The network layer can ask specific repeaters to swap, because it's the first layer that has information about the entire network.

The network layer can also ask repeaters to create links with a given minimum fidelity.

In our simulations, the network layer will also contain a GUI object and a globalState object that together show graphically what the global state of the network is. Note that this information is only available to us because we are running a simulation. In realiy, we practically never know exactly the state we are using, unless we do a lot of measurements to try to figure it out. This is called [quantum state tomography](https://en.wikipedia.org/wiki/Quantum_tomography).

In [11]:
%%writefile repeaterChain.py
%load_ext autoreload
%autoreload 2

from ../4_The_Link_Layer import repeater cable
import globalState

class repeaterChain(object):
    def __init__(self, length):
        self.length = length
        self.repeaters = [repeater(self) for i in range(length)]
#         self.connectedEndnodes = []
        # connect the repeaters with cables
        for i in range(length):
            # for every repeater create a new cable to the right
            if i < length-1:
                self.repeaters[i].rightCable = cable(self.repeaters[i], self.repeaters[i+1])
            if i > 0:
                self.repeaters[i].leftCable = self.repeaters[i-1].rightCable

    def attemptSwap(self, repeater):
        #ask repeater to do a swap
        repeater.attemptSwap(#specify the links to swap#)

    def attemptLinkCreation(self, repeater1, repeater2):
        repeater1.attemptLinkCreation(repeater2)

    def connect(self, endnode): #endnode is a link layer object
        if self.repeaters[0].leftCable == None: # in the future choose where to connect in a better way
            endnode.cable = cable(endnode, self.repeaters[0])
            self.repeaters[0].leftCable = endnode.cable
        else:
            self.repeaters[self.length-1].rightCable = cable(self.repeaters[self.length-1], endnode)
            endnode.cable = self.repeaters[self.length-1].rightCable
        self.assignNetworkId(endnode)

    def assignNetworkId(self, node):
        if type(node) == "endnode":
            if node.cable == None:
                print("endnode is not wired to network.")
            elif node.cable == self.repeaters[0].leftCable:
                node.netId = 0
            elif node.cable == self.repeaters[self.length-1].rightCable:
                node.netId = self.length
        elif type(node) == "repeater":
            if node.rightCable == None and node.leftCable == None:
                print("repeater is not wired to network")
            else:
                node.netId = self.repeaters.index(node) + 1
        else:
            print("unknown node type.")

Overwriting repeaterChain.py


The function `assignNetworkIds` assigns Ids that identify the different link layer objects connected to the network. They are analogous to IP addresses in the classical internet. They are location-dependent: a repeater or an endnode with a smaller id is located more to the left.